In [1]:
import numpy_financial as npf
import datetime
def calculate_ytm(coupon, face_value, years_to_maturity, price, coupon_freq):
    coupon = coupon / 100.0
    periods = years_to_maturity *coupon_freq
    ytm = npf.rate(nper=periods, pmt=coupon*face_value/coupon_freq, pv=-price, fv=face_value)* coupon_freq
    return ytm * 100.0

def calculate_accrued_interest(coupon:float, face_value:float, years_to_maturity:float, payment_frequency:int):
    coupon = coupon / 100.0
    periods = int(years_to_maturity * payment_frequency)
    days_in_period = int(365 / payment_frequency)
    days_since_last_coupon = datetime.datetime.now().timetuple().tm_yday % days_in_period
    coupon_payment = (face_value * coupon) / payment_frequency
    accrued_interest = coupon_payment * days_since_last_coupon / days_in_period
    return accrued_interest

def calculate_dirty_price(clean_price:float, accrued_interest:float):
    dirty_price = clean_price + accrued_interest
    return dirty_price

def  calculate_time_to_maturity(maturity:datetime):
    return (maturity.date() - datetime.date.today()).days / 365

In [2]:
bond_source_url = 'https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc'


In [3]:
import pandas as pd

# Set the URL to extract the data from
url = bond_source_url
# Use pandas to extract the tables from the URL
tables = pd.read_html(url)
# Select the first table, which contains the bond data
bond_data = tables[0]
# Print the first 10 rows of the bond data
print(bond_data.head(10))


                                              Issuer  Coupon (%)  \
0  Treasury 6% 07/12/2028 GBP | GB0002404191 | 02...        6.00   
1  Treasury 5% 07/03/2025 GBP | GB0030880693 | 30...        5.00   
2  Treasury 4.75% 07/12/2030 GBP | GB00B24FF097 |...        4.75   
3  Treasury 4.75% 07/12/2038 GBP | GB00B00NY175 |...        4.75   
4  Treasury 4.5% 07/09/2034 GBP | GB00B52WS153 | ...        4.50   
5  Treasury 4.5% 07/12/2042 GBP | GB00B1VWPJ53 | ...        4.50   
6  Treasury 4.25% 07/06/2032 GBP | GB0004893086 |...        4.25   
7  Treasury 4.25% 07/09/2039 GBP | GB00B3KJDS62 |...        4.25   
8  Treasury 4.25% 07/12/2049 GBP | GB00B39R3707 |...        4.25   
9  Treasury 4.25% 07/12/2027 GBP | GB00B16NNR78 |...        4.25   

           Maturity    Price                  Actions  
0   7 December 2028  112.520  View factsheet Deal now  
1      7 March 2025  101.980  View factsheet Deal now  
2   7 December 2030  108.210  View factsheet Deal now  
3   7 December 2038  109.87

In [4]:
bond_data.columns


Index(['Issuer', 'Coupon (%)', 'Maturity', 'Price', 'Actions'], dtype='object')

In [5]:
import pandas as pd

url = "https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/uk-gilts?column=coupon&order=desc"

bond_data = pd.read_html(url)[0]

bond_data['ShortName'] = bond_data['Issuer'].apply(lambda x: x.split('|')[1].strip())
bond_data["Maturity"] = pd.to_datetime(bond_data["Maturity"], format="%d %B %Y")
bond_data['Ttm'] = bond_data["Maturity"].apply(lambda maturity: calculate_time_to_maturity(maturity))
bond_data['CouponFreq']=2
bond_data=bond_data.rename(columns={'Coupon (%)': 'Coupon'})
bond_data=bond_data.drop(columns=['Issuer','Actions'])
col = bond_data.pop('ShortName')
bond_data.insert(0, 'ShortName', col)

print(bond_data.head())


      ShortName  Coupon   Maturity   Price        Ttm  CouponFreq
0  GB0002404191    6.00 2028-12-07  112.52   5.649315           2
1  GB0030880693    5.00 2025-03-07  101.98   1.893151           2
2  GB00B24FF097    4.75 2030-12-07  108.21   7.649315           2
3  GB00B00NY175    4.75 2038-12-07  109.87  15.654795           2
4  GB00B52WS153    4.50 2034-09-07  107.47  11.402740           2


In [6]:
bond_data.head

<bound method NDFrame.head of        ShortName  Coupon   Maturity    Price        Ttm  CouponFreq
0   GB0002404191   6.000 2028-12-07  112.520   5.649315           2
1   GB0030880693   5.000 2025-03-07  101.980   1.893151           2
2   GB00B24FF097   4.750 2030-12-07  108.210   7.649315           2
3   GB00B00NY175   4.750 2038-12-07  109.870  15.654795           2
4   GB00B52WS153   4.500 2034-09-07  107.470  11.402740           2
..           ...     ...        ...      ...        ...         ...
59  GB00BL68HJ26   0.125 2026-01-30   90.970   2.794521           2
60  GB00B4JYZV64   0.000 2034-09-07   74.100  11.402740           2
61  GB0030884786   0.000 2025-03-07   92.800   1.893151           2
62  GB00B1HYR000   0.000 2027-12-07   84.400   4.646575           2
63  GB0009140269   0.000 2029-12-07   82.372   6.649315           2

[64 rows x 6 columns]>

In [7]:
bond_data.columns


Index(['ShortName', 'Coupon', 'Maturity', 'Price', 'Ttm', 'CouponFreq'], dtype='object')

In [8]:
FACE_VALUE=100.0

In [9]:
bond_data.head(1)

,ShortName,Coupon,Maturity,Price,Ttm,CouponFreq
0,GB0002404191,6.0,2028-12-07,112.52,5.649315,2


In [10]:
bond_data['AccruedInterest']=bond_data.apply(lambda x: calculate_accrued_interest(x['Coupon'],FACE_VALUE,x['Ttm'],x['CouponFreq']),axis=1)
bond_data['DirtyPrice']=bond_data.apply(lambda x: calculate_dirty_price(x['Price'],x['AccruedInterest']),axis=1)
bond_data['DirtyYield'] = bond_data.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['DirtyPrice'],x['CouponFreq']), axis=1)
bond_data['CleanYield'] = bond_data.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['Price'],x['CouponFreq']), axis=1)

In [11]:
bond_data.dtypes


ShortName                  object
Coupon                    float64
Maturity           datetime64[ns]
Price                     float64
Ttm                       float64
CouponFreq                  int64
AccruedInterest           float64
DirtyPrice                float64
DirtyYield                float64
CleanYield                float64
dtype: object

In [12]:
bond_data=bond_data.sort_values('Ttm')


In [13]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

bond_data_styler=bond_data.style.set_properties(**{'font-size': '6pt'})
bond_data_styler

,ShortName,Coupon,Maturity,Price,Ttm,CouponFreq,AccruedInterest,DirtyPrice,DirtyYield,CleanYield
44,GB00BF0HZ991,0.750000,2023-07-22 00:00:00,99.070000,0.265753,2,0.218407,99.288407,3.463102,4.307098
25,GB00B7Z53659,2.250000,2023-09-07 00:00:00,99.220000,0.394521,2,0.655220,99.875220,2.569917,4.264767
55,GB00BMGR2791,0.125000,2024-01-31 00:00:00,96.900000,0.794521,2,0.036401,96.936401,4.083014,4.131266
40,GB00BFWFPL34,1.000000,2024-04-22 00:00:00,96.810000,1.019178,2,0.291209,97.101209,3.929423,4.230928
24,GB0008983024,2.500000,2024-07-17 00:00:00,374.410000,1.254795,2,0.728022,375.138022,-80.853742,-80.761047
22,GB00BHBFH458,2.750000,2024-09-07 00:00:00,98.110000,1.397260,2,0.800824,98.910824,3.555942,4.156307
52,GB00BLPK7110,0.250000,2025-01-31 00:00:00,93.470000,1.797260,2,0.072802,93.542802,4.009693,4.053961
1,GB0030880693,5.000000,2025-03-07 00:00:00,101.980000,1.893151,2,1.456044,103.436044,3.116836,3.904818
61,GB0030884786,0.000000,2025-03-07 00:00:00,92.800000,1.893151,2,0.000000,92.800000,3.986252,3.986252
27,GB00BTHH2R79,2.000000,2025-09-07 00:00:00,96.090000,2.397260,2,0.582418,96.672418,3.458372,3.719946


In [14]:
bond_data.to_excel(r"c:\temp\bond_data.xlsx", index=False)

In [15]:
# yield filtering 
df_less_than_Ttm = bond_data[bond_data['Ttm'] < 5]

In [16]:
df_less_than_Ttm=df_less_than_Ttm.sort_values("DirtyYield", ascending=False)
cols = ['ShortName', 'Coupon', 'Maturity', 'Price', 'Ttm','DirtyPrice', 'DirtyYield', 'CleanYield']
df_less_than_Ttm = df_less_than_Ttm[cols]
df_less_than_Ttm


,ShortName,Coupon,Maturity,Price,Ttm,DirtyPrice,DirtyYield,CleanYield
55,GB00BMGR2791,0.125,2024-01-31,96.900,0.794521,96.936401,4.083014,4.131266
52,GB00BLPK7110,0.250,2025-01-31,93.470,1.797260,93.542802,4.009693,4.053961
61,GB0030884786,0.000,2025-03-07,92.800,1.893151,92.800000,3.986252,3.986252
40,GB00BFWFPL34,1.000,2024-04-22,96.810,1.019178,97.101209,3.929423,4.230928
62,GB00B1HYR000,0.000,2027-12-07,84.400,4.646575,84.400000,3.683571,3.683571
49,GB00BNNGP668,0.375,2026-10-22,89.130,3.520548,89.239203,3.660602,3.696238
17,GB00BPCJD880,3.500,2025-10-22,98.820,2.520548,99.839231,3.567261,3.996788
22,GB00BHBFH458,2.750,2024-09-07,98.110,1.397260,98.910824,3.555942,4.156307
59,GB00BL68HJ26,0.125,2026-01-30,90.970,2.794521,91.006401,3.533109,3.547701
56,GB00BMBL1G81,0.125,2028-01-31,85.140,4.797260,85.176401,3.509385,3.518479
